In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [2]:
BaseOptions = mp.tasks.BaseOptions
FaceDetector = mp.tasks.vision.FaceDetector
FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceDetectorOptions(
    base_options=BaseOptions(model_asset_path='model/blaze_face_short_range.tflite'),
    running_mode=VisionRunningMode.VIDEO
)

with FaceDetector.create_from_options(options) as detector:
    cap = cv2.VideoCapture(1)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Cannot receive frame.")
            break
        
        frame = cv2.resize(frame, (960, 540))
        h, w, _ = frame.shape
        
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        detection_result = detector.detect_for_video(mp_image, int(cv2.getTickCount()))

        for detection in detection_result.detections:
            bbox = detection.bounding_box
            lx, ly = int(bbox.origin_x), int(bbox.origin_y)
            width, height = int(bbox.width), int(bbox.height)
            cv2.rectangle(frame, (lx, ly), (lx + width, ly + height), (0, 0, 255), 2)

            for keypoint in detection.keypoints:
                cx = int(keypoint.x * w)
                cy = int(keypoint.y * h)
                cv2.circle(frame, (cx, cy), 4, (255, 0, 0), -1)

        cv2.imshow('Face Detection (Press ESC to exit)', frame)
        
        if cv2.waitKey(1) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()